# Setup: dependencies, global variables, functions, etc.

In [1]:
import ant_demo

In [40]:
import Analysis_Demo
from importlib import reload
reload(Analysis_Demo)
from Analysis_Demo import * # functions and global variables, like data_dir
import mne
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
import os
import fooof

mne.set_log_level(verbose=0)

%matplotlib qt

close_plot = False

In [3]:
# Loop through .CNT files in the directory
for filename in os.listdir(parent_dir):
    # Only look at the .CNT files
    if not filename.endswith('.cnt'): continue
    if not filename.endswith('18.cnt'): continue
    # Open the data as an mne.Raw object
    raw_unfilt = load_CNT(os.path.join(parent_dir, filename))

Re-referenced to pseudo-mastoids
Channel positions loaded for WG_Net_NA-261
Annotations (events) renamed
Eyes open/closed recoded as single annotations with durations of ~20s


# Independent components analysis

In [ ]:
ica = mne.preprocessing.ICA()
raw = raw_unfilt.copy().filter(2,30).resample(sfreq=128)
raw.info['bads'] = [ch for ch,noisy in zip(raw.ch_names, raw.get_data().std(axis=1) > .00008) if noisy]
raw_unfilt.info['bads'] = raw.info['bads']
ica.fit(raw)

In [ ]:
ica.plot_components(range(15))
ica.plot_sources(raw, range(15))

In [ ]:
bad_ics = [0,4,12]
ica.plot_overlay(raw, exclude=bad_ics)

In [ ]:
raw_ica = ica.apply(raw_unfilt.copy(), exclude=bad_ics)

In [ ]:
raw_unfilt.plot()

# Frequency domain analysis: Eyes open/closed alpha power

## Power spectrum of the raw data
Look for peaks at 60Hz and its harmonics. How clean are the data?  
Look for peaks arund 10Hz. Those are your alpha oscillations!

In [5]:
# Filter for clarity
raw = raw_unfilt.copy().filter(1,30).notch_filter(60)

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    1.3s finished


In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)
raw_unfilt.plot_psd(fmax=150, show=True, ax=ax1)
raw.plot_psd(fmax=150, show=True, ax=ax2)
ax1.set(title='Raw unfiltered data')
ax2.set(title='Filtered data (1-30Hz bandpass, 60Hz notch)')
clean_up_plot(fig, 'Power spectrum of full dataset.png', close_plot)

## View the event markers

In [ ]:
raw_temp = raw.copy()
raw_temp.annotations.delete([idx for idx, desc in enumerate(raw.annotations.description) if desc in ['exp_start','exp_stop','impedance']])
fig = mne.viz.plot_events(mne.events_from_annotations(raw_temp)[0], event_id=mne.events_from_annotations(raw_temp)[1], sfreq=raw_temp.info['sfreq'])
clean_up_plot(fig, 'Event markers.png',close_plot)

In [ ]:
annot = raw.annotations
raw.annotations.delete([idx for idx, desc in enumerate(raw.annotations.description) if desc in ['exp_start','exp_stop','impedance']])
fig = mne.viz.plot_events(mne.events_from_annotations(raw)[0], event_id=mne.events_from_annotations(raw)[1], sfreq=raw.info['sfreq'])
raw.set_annotations(annot)
clean_up_plot(fig, 'Event markers.png', close_plot)

## Plot PSD for eyes open/closed, separately

In [6]:
psds = {cond: 
        mne.concatenate_raws(
            raw.crop_by_annotations(
                annotations=[a for a in raw.annotations if a['description'] == cond]
            )).compute_psd(method='welch', fmin=1, fmax=60, n_per_seg=int(raw.info['sfreq']))
        for cond in ['eyes_open','eyes_closed']}

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)
psds['eyes_open'].plot(axes=ax1)
psds['eyes_closed'].plot(axes=ax2)
ax1.set(title='Eyes Open')
ax2.set(title='Eyes Closed')
clean_up_plot(fig, 'Power spectrum eyes open vs.closed with MNE.png', close_plot)

In [ ]:
bands = {'Delta (0-4 Hz)': (0, 4),'Theta (4-8 Hz)': (4, 8),'Alpha (8-12 Hz)': (8, 12), 'Beta (12-30 Hz)': (12, 30)}
fig, axs = plt.subplots(2,4)
psds['eyes_open'].plot_topomap(bands=bands, axes=axs[0,0:4], cmap='RdBu_r')
psds['eyes_closed'].plot_topomap(bands=bands, axes=axs[1,0:4], cmap='RdBu_r')
fig.suptitle('Frequency band maps: Eyes open (top) and Eyes closed (bottom)')
clean_up_plot(fig, 'Frequency mapping eyes open vs.closed with MNE.png', close_plot)

In [ ]:
df = pd.concat({k:v.to_data_frame(long_format=True, index='freq') for k,v in psds.items()})
df = df.reset_index().drop(columns='ch_type').rename(columns={'value':'power', 'level_0':'condition'})
df['power (log)'] = np.log(df['power'])

fig = sn.relplot(df, kind='line', x='freq', y='power (log)', hue='condition')
clean_up_plot(fig.figure, 'Average power spectrum eyes open vs. closed.png', close_plot)
fig = sn.relplot(df, kind='line', x='freq', y='power (log)', hue='condition', style='channel', alpha=.5)
clean_up_plot(fig.figure, 'Power spectrum eyes open vs. closed.png', close_plot)
fig = sn.relplot(df, kind='line', x='freq', y='power (log)', hue='condition', col='channel', col_wrap=8)
clean_up_plot(fig.figure, 'Power spectrum eyes open vs. closed - per channel.png', close_plot)

In [ ]:
# Look at the peak alpha frequency per condition X channel
df_peaks = df.query("7<freq<13").groupby(['condition','channel']).apply(lambda x:x.loc[x['power (log)'].idxmax(),'freq']).rename('freq')
print(type(df_peaks))
fig = sn.catplot(df_peaks.reset_index(), kind='bar', x='freq', y='condition', col='channel',col_wrap=8, sharex=False)
clean_up_plot(fig.figure, 'Peak alpha - per channel.png', close_plot)
df_peaks.groupby(['channel']).apply(lambda x: x.diff()['eyes_open']).hist(bins=50)

In [7]:
import fooof
fms = {k:fooof.FOOOFGroup() for k in psds.keys()}
for k,v in psds.items():
    fms[k].fit(freqs=v.freqs, power_spectra=v.get_data())
df = pd.DataFrame({
    'eo':fms['eyes_open'].to_df(fooof.Bands({'alpha': [8,12]}))['alpha_cf'],
    'ec':fms['eyes_closed'].to_df(fooof.Bands({'alpha': [8,12]}))['alpha_cf'],
})
df['open_bias'] = df['eo']-df['ec']
df['open_bias'].hist(bins=50)

Running FOOOFGroup across 63 power spectra.
Running FOOOFGroup across 63 power spectra.


<Axes: >

# ERP analysis: oddball P300

In [8]:
raw_ob = raw_unfilt.copy().filter(.1,30).notch_filter(60)

epochs = mne.Epochs(
    raw_ob, 
    events = mne.events_from_annotations(raw_ob,
                                         event_id={
                                             'frequent': 1,
                                             'rare': 2,
                                         }
                                        )[0],
    event_id = {'frequent':1, 'rare':2},
    tmin = -0.2, tmax = .8, baseline = (-0.2, 0),
)

epochs.drop(epochs.to_data_frame(index=['time','condition','epoch']).groupby('epoch').std().max(axis=1).sort_values(ascending=False).index[0:int(.05*len(epochs))])
evoked = epochs.average(by_event_type=True)

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    1.7s finished
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    1.4s finished


In [11]:
fig = mne.viz.plot_compare_evokeds(evoked, 
                                   picks=[ch for ch in raw_ob.ch_names if ((ch in [
                                       '5L','6L','7L','8L','9L',
                                       '5Z','6Z','7Z','8Z','9Z',
                                       '5R','6R','7R','8R','9R',
                                   ]))],
                                   show_sensors=True,
                                   combine='mean',
                                   title='P300'
                                  )
clean_up_plot(fig[0], 'P300_averaged.png', close_plot)

In [10]:
fig = evoked[1].plot_joint(title='Response to Rare Stimuli', times=[0,.100,.200,.300,.400,.500,.600])
clean_up_plot(fig, 'Response to Rare Stimuli_v2.png', close_plot)

In [ ]:
tf = raw.compute_tfr('morlet',np.arange(6,26), picks='11R', decim=10, output='power')

In [ ]:
tf.plot(dB=True,baseline=[0,20])

In [77]:
# T-test for SNR
from scipy.stats import ttest_ind
evoked_diff = mne.combine_evoked(evoked, [-1,1])
ch_best,latency = evoked_diff.get_peak(tmin=.300,tmax=.500)
df_windowed = epochs.to_data_frame(index=['time','condition','epoch']).query("-.1<time-@latency<.1")
df_auc = df_windowed.groupby(['epoch','condition']).mean()
df_t = pd.Series(index=pd.Index(name='channel', data=evoked_diff.ch_names),
                 name='SNR',
                 data=ttest_ind(df_auc.query("condition=='rare'"), df_auc.query("condition=='frequent'"), equal_var=False).statistic,
                )

-frequent + rare


In [100]:
df_auc.melt(ignore_index=False, value_name='auc').set_index('channel', append=True)

,,,auc
epoch,condition,channel,
0,rare,1Z,0.730630
1,frequent,1Z,-19.547240
2,frequent,1Z,6.105339
3,rare,1Z,3.965618
4,frequent,1Z,2.257274
...,...,...,...
155,frequent,5Z,8.160531
156,frequent,5Z,18.483575
157,frequent,5Z,13.002523


In [95]:
df_t

channel
1Z     1.274959
2Z     1.284167
3Z     1.557130
4Z     1.617737
6Z     4.532384
         ...   
5RC    1.703978
1RD   -0.006102
3RD   -0.710747
4RD   -0.047266
5Z     2.291669
Name: SNR, Length: 63, dtype: float64